In [1]:
# import tensorflow as tf
# from keras import backend as K

# num_cores = 4
# CPU = True
# GPU = False

# if GPU:
#     num_GPU = 1
#     num_CPU = 1
# if CPU:
#     num_CPU = 1
#     num_GPU = 0

# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
#         inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
#         device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
# session = tf.Session(config=config)
# K.set_session(session)

Using TensorFlow backend.


In [80]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import *
import matplotlib.pyplot as plt
from tokenizer import tokenizer as tweet_tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import * 
from keras.regularizers import *
from keras.models import load_model
import pickle


%matplotlib inline

PATH = '/home/epita/sim/sentiment_analysis/data/'

path_train_3 = f'{PATH}/data_train_3.csv'
path_test_3 = f'{PATH}/data_test_3.csv'
path_train_7 = f'{PATH}/data_train_7.csv'
path_train_16m_3 = f'{PATH}/training.1600000.processed.noemoticon.csv'

## One hot sample

In [16]:
text = 'The quick brown fox jumped over the lazy dog.'

words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)

result = one_hot(text, round(vocab_size*1.3))
print(result)

8
[6, 2, 7, 1, 4, 4, 6, 6, 7]


## Tweet Tokenizer sample

In [60]:
T = tweet_tokenizer.TweetTokenizer(preserve_case=False, preserve_url=False)

text = 'The quick brown fox jumped over the lazy dog.'

T.tokenize(text)

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', '.']

## Lemmatizer / Stemmer samples

In [22]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
 
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
 
print("Stem %s: %s" % ("studying", stemmer.stem("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying", pos="v")))

Stem studying: studi
Lemmatise studying: studying
Lemmatise studying: study


In [33]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.corpus import wordnet as wn


tag_map = defaultdict(lambda: wn.NOUN, {'J': wn.ADJ, 'V': wn.VERB, 'R': wn.ADV})

s = "This is a simple sentence that would be allowing us to try lemmatizing"

tokens = word_tokenize(s)
tokens_pos = pos_tag(tokens)
 
print(tokens_pos)


lems = [lemmatiser.lemmatize(word, tag_map[pos[0]]) for word, pos in tokens_pos]
print(lems)

[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('simple', 'JJ'), ('sentence', 'NN'), ('that', 'WDT'), ('would', 'MD'), ('be', 'VB'), ('allowing', 'VBG'), ('us', 'PRP'), ('to', 'TO'), ('try', 'VB'), ('lemmatizing', 'VBG')]
['This', 'be', 'a', 'simple', 'sentence', 'that', 'would', 'be', 'allow', 'u', 'to', 'try', 'lemmatizing']


# Serious business

## Preprocess Train_16m_3

In [19]:
tweets_16m = pd.read_csv(train_16m_3, sep=',', encoding='latin-1').drop(columns=['date', 'flag', 'user'])

print(tweets_16m.shape)
tweets_16m.sample(5)

(1600000, 3)


,target,id,text
1136298,4,1976586207,@Doc_Sprocket sorry - i thought i wasn't going...
82391,0,1753038955,horrible headache
883382,4,1686122324,YES! ... finished my assignment! 1 down and 3 ...
1473925,4,2065642051,best #kiss i ever seen http://bit.ly/kdepj
795603,0,2327426618,"thinking she failed her geo exam, and possibly..."


In [20]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets_16m['target'] = tweets_16m['target'].apply(lambda x: {0: 0, 2: 1, 4: 2}[x])  # Neg (0), Neu (2), Pos (4)
tweets_16m['text'] = tweets_16m['text'].apply(lambda x: standardization(x))

In [21]:
display(tweets_16m.sample(5))
tweets_16m.sample(5)['text'].iloc[0]

,target,id,text
174553,0,1964566165,babe u aint read tweet phone fall apart new co...
235416,0,1979777211,not cold go get
901553,2,1694256723,get new frens-frm germany mexico n france havi...
913630,2,1752671998,im not cookie wont comment heh girl sing crush...
832154,2,1557553994,ah sunday morning nice day get realize money a...


'night twitterverse seek ever elusive prince sleep sweet dream u good morning ever apply zzzz'

In [23]:
tweets_16m.to_feather('pickles/train_16m_3.feather')

## Preprocess Train_3

In [50]:
tweets = pd.read_csv(path_train_3, sep='\t', names=['ID', 'Class', 'Tweet'])
tweets_7 = pd.read_csv(path_train_7, sep='\t', names=['ID', 'Class', 'Tweet'], dtype={'Tweet': str})
tweets_3_test = pd.read_csv(path_test_3, sep='\t', names=['ID', 'Class', 'Tweet'], dtype={'Tweet': str})

tweets.shape, tweets_7.shape, tweets_3_test.shape

((50333, 3), (1630, 3), (1630, 3))

In [5]:
tweets.sample(5)

,ID,Class,Tweet
24563,630258702054916096,neutral,y'all are gonna get spammed af on tuesday for ...
11995,238557845359038465,neutral,@LawsonRyan AND you were played on Styled To R...
26247,632984127277539328,neutral,Paper Towns is released in the UK tomorrow and...
26208,632971809802047488,positive,@NGfootyactivist kindly Retweet my last tweet ...
5374,172548568329093120,neutral,See John Lloyd Cruz & Angel Locsin at the \Uno...


In [31]:
tweets_7.head(5)

,ID,Class,Tweet
0,0,0,@liamch88 yeah! :) playing well
1,1,0,At least I don't have a guy trying to discoura...
2,2,0,UPLIFT: If you're still discouraged it means y...
3,3,0,"...at your age, the heyday in the blood is tam..."
4,4,-2,i was so embarrassed when she saw us i was lik...


In [20]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets['Sentiment'] = tweets['Class'].apply(lambda x: {'negative': 0, 'neutral': 1, 'positive': 2}[x])
tweets['Tweet'] = tweets['Tweet'].apply(lambda x: standardization(x))

In [32]:
tweets_7['Sentiment'] = tweets_7['Class']
tweets_7['Tweet'] = tweets_7['Tweet'].apply(lambda x: standardization(x))

In [33]:
tweets_3_test['Sentiment'] = tweets_3_test['Class'].apply(lambda x: {'negative': 0, 'neutral': 1, 'positive': 2}[x])
tweets_3_test['Tweet'] = tweets_3_test['Tweet'].apply(lambda x: standardization(x))

In [34]:
display(tweets.sample(2))
display(tweets_7.head(2))
display(tweets_3_test.head(2))

,ID,Class,Tweet
28257,635721210806956032,neutral,For an art project in 7th grade we had to make...
17615,634516378268119041,negative,.@marklevinshow apparently Rick Perry is an id...


,ID,Class,Tweet,Sentiment
0,0,0,yeah ☺ ️ playing well,0
1,1,0,least not guy try discourage anymore want neve...,0


,ID,Class,Tweet,Sentiment
0,449,negative,site crash everytime try book help tell nothin...,0
1,450,negative,theme week ask lord strength perspective perse...,0


In [35]:
tweets_7.to_feather('pickles/train_7.feather')

In [36]:
tweets_3_test.to_feather('pickles/test_3.feather')

In [22]:
tweets.to_feather('pickles/train_3.feather')

## Training

In [60]:
tweets = pd.read_feather('pickles/train_3.feather')
test_3 = pd.read_feather('pickles/test_3.feather')
train_7 = pd.read_feather('pickles/train_7.feather')

train_x, train_y = tweets['Tweet'], tweets['Sentiment']
test_x, test_y = test_3['Tweet'], test_3['Sentiment']
train7_x, train7_y = train_7['Tweet'], train_7['Sentiment']

/home/epita/.local/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


* Fitting the tokenizer on our entire tweet base to make sure the indexes are rights

In [81]:
all_tweets = pd.concat([train_x, test_x, train7_x])
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweets)
word_index = tokenizer.word_index

pickle.dump(tokenizer, open("pickles/tokenizer.pickle", "wb"))

In [82]:
tokenizer = pickle.load(open("pickles/tokenizer.pickle", 'rb'))

* Looking for the MAX_SEQUENCE_LENGTH in all our sequences:

In [84]:
train_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)
train7_sequences = tokenizer.texts_to_sequences(train7_x)

sequences = train_sequences + test_sequences + train7_sequences
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

MAX_SEQUENCE_LENGTH

32

* We only train on train_3:

In [86]:
train_sequences = pad_sequences(train_sequences, MAX_SEQUENCE_LENGTH)
train_sequences.shape

(50333, 32)

#### Loading Datastories's embeddings

In [87]:
# googlenews_w2v = KeyedVectors.load_word2vec_format('data/embeddings/GoogleNews-vectors-negative300.bin', binary=True)
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import get_tmpfile


tmp_file = get_tmpfile('datastories.300d.word2vec')
glove2word2vec('data/embeddings/datastories.twitter.300d.txt', tmp_file)
w2v = KeyedVectors.load_word2vec_format(tmp_file)

In [89]:
targets = to_categorical(train_y, 3)
nb_words = len(word_index) + 1

EMBEDDING_DIM = 300
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

oov = []  # Out of vocabulary
oov.append((np.random.rand(EMBEDDING_DIM) * 2.0) - 1.0)
oov = oov / np.linalg.norm(oov)

print(oov.shape)

for word, i in word_index.items():
    if word in w2v.vocab:
        embedding_matrix[i] = w2v.word_vec(word)
    else:
        embedding_matrix[i] = oov

print(embedding_matrix.shape)

(1, 300)
(37785, 300)


In [90]:
# Claim memory back from this very large object we don't use anymore
del w2v

In [91]:
X_train, X_val, y_train, y_val = train_test_split(train_sequences, targets, test_size=0.3)

print('training set: ' + str(len(X_train)) + ' samples')
print('validation set: ' + str(len(X_val)) + ' samples')

print('x_train:', X_train.shape)
print('y_train:', y_train.shape)

training set: 35233 samples
validation set: 15100 samples
x_train: (35233, 32)
y_train: (35233, 3)


In [92]:
def model_2017():
    vocab_size = embedding_matrix.shape[0]
    embedding_size = embedding_matrix.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
    model.add(GaussianNoise(0.3))
    # model.add(SpatialDropout1D(0.3))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, recurrent_dropout=0.3, return_sequences=True)))
    # model.add(Dropout(0.5))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, recurrent_dropout=0.3, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(3, activity_regularizer=l2(1e-4)))
    model.add(Activation('softmax'))

    # RMSprop or lr=1e-3
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-2), metrics=['acc'])

    print(model.summary())
    
    return model

def model_mine():
    vocab_size = embedding_matrix.shape[0]
    embedding_size = embedding_matrix.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    print(model.summary())
    
    return model

In [93]:
model = model_mine()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
__________

In [94]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=5)

Train on 35233 samples, validate on 15100 samples
Epoch 1/5
35233/35233 [==============================] - 129s 4ms/step - loss: 0.8453 - acc: 0.5889 - val_loss: 0.7882 - val_acc: 0.6365
Epoch 2/5
35233/35233 [==============================] - 124s 4ms/step - loss: 0.7753 - acc: 0.6378 - val_loss: 0.7691 - val_acc: 0.6319
Epoch 3/5
35233/35233 [==============================] - 127s 4ms/step - loss: 0.7457 - acc: 0.6527 - val_loss: 0.7332 - val_acc: 0.6591
Epoch 4/5
35233/35233 [==============================] - 126s 4ms/step - loss: 0.7211 - acc: 0.6665 - val_loss: 0.7522 - val_acc: 0.6555
Epoch 5/5
35233/35233 [==============================] - 126s 4ms/step - loss: 0.7030 - acc: 0.6776 - val_loss: 0.7273 - val_acc: 0.6644


In [99]:
test_sequences = pad_sequences(test_sequences, MAX_SEQUENCE_LENGTH)
test_sequences.shape
Y_test = to_categorical(test_y, 3)

In [101]:
model.evaluate(test_sequences, Y_test, batch_size=128)

1630/1630 [==============================] - 2s 1ms/step


[1.0056506370474225, 0.5263803679518905]

In [102]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=7, initial_epoch=5)

Train on 35233 samples, validate on 15100 samples
Epoch 6/7
35233/35233 [==============================] - 128s 4ms/step - loss: 0.6775 - acc: 0.6945 - val_loss: 0.7299 - val_acc: 0.6666
Epoch 7/7
35233/35233 [==============================] - 125s 4ms/step - loss: 0.6517 - acc: 0.7067 - val_loss: 0.7582 - val_acc: 0.6548


In [103]:
model.evaluate(test_sequences, Y_test, batch_size=128)

1630/1630 [==============================] - 2s 1ms/step


[1.1275912613956476, 0.5128834352902839]

In [104]:
model.save('pickles/train_3_65-mine.h5')

In [16]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=5)

Train on 35233 samples, validate on 15100 samples
Epoch 1/5
35233/35233 [==============================] - 135s 4ms/step - loss: 0.8453 - acc: 0.5898 - val_loss: 0.7717 - val_acc: 0.6374
Epoch 2/5
35233/35233 [==============================] - 133s 4ms/step - loss: 0.7761 - acc: 0.6362 - val_loss: 0.7497 - val_acc: 0.6522
Epoch 3/5
35233/35233 [==============================] - 131s 4ms/step - loss: 0.7480 - acc: 0.6517 - val_loss: 0.7484 - val_acc: 0.6479
Epoch 4/5
35233/35233 [==============================] - 134s 4ms/step - loss: 0.7245 - acc: 0.6649 - val_loss: 0.7320 - val_acc: 0.6603
Epoch 5/5
35233/35233 [==============================] - 131s 4ms/step - loss: 0.7048 - acc: 0.6773 - val_loss: 0.7240 - val_acc: 0.6668


In [17]:
# model.save('pickles/train_3_66.7-emb_dr3_bi_dr5_bi_dr4.h5')

In [24]:
model = load_model('pickles/train_3_66.7-emb_dr3_bi_dr5_bi_dr4.h5')

In [25]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=7, initial_epoch=5)

Train on 35233 samples, validate on 15100 samples
Epoch 6/7
35233/35233 [==============================] - 140s 4ms/step - loss: 0.6850 - acc: 0.6889 - val_loss: 0.7250 - val_acc: 0.6699
Epoch 7/7
35233/35233 [==============================] - 134s 4ms/step - loss: 0.6630 - acc: 0.7009 - val_loss: 0.7242 - val_acc: 0.6721


In [26]:
# model.save('pickles/train_3_67.2-emb_dr3_bi_dr5_bi_dr4.h5')

In [27]:
loss, acc = model.evaluate(X_val, y_val)

print(f'val_loss: {loss}\nval_acc: {acc}')

15100/15100 [==============================] - 88s 6ms/step
val_loss: 0.7242394660956023
val_acc: 0.6721192052664346


### Bi-derectional LSTM x2

In [20]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=20)

Train on 35233 samples, validate on 15100 samples
Epoch 1/20
35233/35233 [==============================] - 133s 4ms/step - loss: 0.8855 - acc: 0.5888 - val_loss: 0.7555 - val_acc: 0.6511
Epoch 2/20
35233/35233 [==============================] - 131s 4ms/step - loss: 0.7755 - acc: 0.6376 - val_loss: 0.7467 - val_acc: 0.6560
Epoch 3/20
35233/35233 [==============================] - 128s 4ms/step - loss: 0.7400 - acc: 0.6575 - val_loss: 0.7229 - val_acc: 0.6686
Epoch 4/20
35233/35233 [==============================] - 131s 4ms/step - loss: 0.7124 - acc: 0.6727 - val_loss: 0.7285 - val_acc: 0.6725
Epoch 5/20
35233/35233 [==============================] - 129s 4ms/step - loss: 0.6925 - acc: 0.6867 - val_loss: 0.7240 - val_acc: 0.6644
Epoch 6/20
35233/35233 [==============================] - 131s 4ms/step - loss: 0.6774 - acc: 0.6938 - val_loss: 0.7288 - val_acc: 0.6676
Epoch 7/20
35233/35233 [==============================] - 122s 3ms/step - loss: 0.6579 - acc: 0.7035 - val_loss: 0.7342 - 

## Transfer Learning

In [31]:
model = load_model('pickles/train_3_67.2-emb_dr3_bi_dr5_bi_dr4.h5')

model.layers.pop()
model.layers.pop()
for layer in model.layers:
    layer.trainable = False
model.add(Dense(150, activation='relu'))
# model.add(Flatten())
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11090400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
__________

In [ ]:
history = model.fit(x_train_7, y_train_7, validation_data=(x_val_7,y_val_7), epochs=11, batch_size=50)

### Logistic Regression

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)

X = cv.fit_transform(tweets['Tweet'])
target = tweets['Sentiment']

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(X, target)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, solver='saga', multi_class='auto')
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6240464081373173
Accuracy for C=0.05: 0.6523363000635728
Accuracy for C=0.25: 0.6594882390336936
Accuracy for C=0.5: 0.6562301335028607
Accuracy for C=1: 0.6511443102352193


#### Best and worst features

In [11]:
feature_to_coef = {word: coef for word, coef in zip(cv.get_feature_names(), lr.coef_[0])}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print (best_positive)
    
print()
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print (best_negative)

('gifford', -1.2377102595086205)
('seinfeld', -1.208227717966838)
('bout', -1.121596723479545)
('bless', -1.1170007079307602)
('greatest', -1.0976803765492371)

('worst', 2.4448066170437612)
('sucks', 2.221116192138116)
('stupid', 2.2174777929515206)
('fucked', 2.1713174119575873)
('fuck', 2.127179056507483)


## Learning (pseudo-code)

In [ ]:
# transfer:

model.layers.pop()
model.layers.pop()
for layer in model.layers:
    layer.trainable = False
Dense(150, 'relu')
Dense(64, 'relu')
Dense(7, 'softmax')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Notes
* Télécharger 50 millions de tweets pour le word2vec
* Metric pour le script du prof: pearson
* Utiliser pearson au lieu d'accuracy
* L'année dernière le prof a eu 78% accuracy 

**Output attendu**: produire un fichier avec les mêmes noms de cartégories sur un fichier de test façon kaggle